In [15]:
from knn import kNN, convert_embeddings_to_tf
import numpy as np
from tqdm import tqdm
from load_mini_imagenet import load_mini_imagenet_data, split_for_n_shot

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
X_test, y_test = load_mini_imagenet_data("mini-imagenet-cache-test.pkl")

In [4]:
X_support, y_support, X_test, y_test = split_for_n_shot(X_test, y_test, n=5)

In [5]:
mean = np.mean(X_support, axis=(0, 1, 2))
stddev = np.std(X_support, axis=(0, 1, 2))

In [6]:
X_support_norm = (X_support - mean) / stddev
X_test_norm = (X_test - mean) / stddev

In [25]:
# extract pixels as features
test_features = X_test.reshape(X_test.shape[0], -1)
support_features = X_support.reshape(X_support.shape[0], -1)
print(test_features.shape)
print(support_features.shape)

(11900, 21168)
(100, 21168)


In [8]:
# extract SIFT features
from sift_extract import extract_sift_for_dataset

test_features = extract_sift_for_dataset(X_test, step_size=7)
support_features = extract_sift_for_dataset(X_support, step_size=7)
print(test_features.shape)
print(support_features.shape)

(11900, 18432)
(100, 18432)


In [9]:
correct = 0
for feat, label in tqdm(zip(test_features, y_test)):
    predicted = kNN(support_features, y_support, feat, k=3)
    if predicted == label:
        correct += 1

11900it [00:38, 310.58it/s]


In [10]:
# pixel, k = 1, euclidean
print(correct / len(test_features))

0.0726890756302521


In [11]:
# pixel, k = 3, euclidean
print(correct / len(test_features))

0.0726890756302521


In [12]:
# SIFT, k = 1, euclidean
print(correct / len(test_features))

0.0726890756302521


In [13]:
# SIFT, k = 3, euclidean
print(correct / len(test_features))

0.0726890756302521


In [26]:
convert_embeddings_to_tf(support_features, y_support, log_dir="logs/pixels/")